# Qumulo API Activity Data exmaples

In [1]:
from qumulo.rest_client import RestClient

In [3]:
# change these to your Qumulo API cluster and credentials
rc = RestClient('<qumulo-cluster>', 8000)
rc.login('<login>', '<password>');

### make the current activity api call that we'll be using below

In [ ]:
activity_data = rc.analytics.current_activity_get()

### Show all of types of activity data

In [21]:
list(set([d['type'] for d in activity_data['entries']]))

[u'file-iops-write',
 u'file-throughput-read',
 u'file-iops-read',
 u'metadata-iops-read',
 u'metadata-iops-write',
 u'file-throughput-write']

### Sum up the rates by activity type

In [38]:
types_data = {}
for d in activity_data['entries']:
    if d['type'] not in types_data:
        types_data[d['type']] = 0
    types_data[d['type']] += d['rate']
for t, val in types_data.iteritems():
    print("%22s - %.1f" % (t, val))

       file-iops-write - 211.3
  file-throughput-read - 240571.7
        file-iops-read - 5.0
    metadata-iops-read - 13.8
   metadata-iops-write - 1.8
 file-throughput-write - 2206720.0


### Report clients by throughput from highest to lowest

In [50]:
ip_data = {}
for d in activity_data['entries']:
    if 'throughput' in d['type']:
        if d['ip'] not in ip_data:
            ip_data[d['ip']] = 0
        ip_data[d['ip']] += d['rate']
for ip, val in sorted(ip_data.items(), key=lambda x: x[1], reverse=True):
    print("%22s - %.1f" % (ip, val))

         10.120.247.92 - 1172616.5
         10.120.247.95 - 746564.3
         10.120.247.91 - 528110.9


### Resolve ids to paths

In [37]:
batch_size = 1000
ids_to_paths = {}

ids = list(set([d['id'] for d in activity_data['entries']]))

for offset in range(0, len(ids), batch_size):
    resolve_data = rc.fs.resolve_paths(ids[offset:offset+batch_size])
    for id_path in resolve_data:
        ids_to_paths[id_path['id']] = id_path['path']

print("Resolved %s ids to paths." % len(ids_to_paths))

Resolved 442 ids to paths.


### Summarize the most active directories by IOPS

In [62]:
directory_levels = 1
path_data = {}

for d in activity_data['entries']:
    path = ids_to_paths[d['id']]
    if path == '':
        path = '/'
    path = '/'.join(path.split('/')[0:directory_levels+1])
    if 'throughput' in d['type']:
        if path not in path_data:
            path_data[path] = 0
        path_data[path] += d['rate']

for path, val in sorted(path_data.items(), key=lambda x: x[1], reverse=True):
    print("%32s - %.1f" % (path, val))

         /qumulo-historical-data - 866304.0
                       /query_db - 553164.8
                       /tv-shows - 528110.9
                               / - 499712.0
